## AIFour: Computer AI Playing Connect Four

### What is Connect Four
Connect Four is a two-player game where the aim is to have four of the same colored disc horizontally, vertically or diagonnally. Players will take turn to place their own colored disc on one of the seven columns in the 7x6 board. The discs of the same column stack on top of each other.

It is a zero-sum game because either player would win, or both would draw the game. It is also a complete game because each player has perfect information on what the current board look like and what are the available moves for the other player.

### What makes Connect Four an interesting AI search problem?
Although there is limited steps that each player can take (at most 7, unless the board has a different number of columns), the number of possible state of the board can be up to four trillion. 

To build a computer bot that can play against a human player, it has to understand the board and determine a move in a reasonably short time. Therefore it is impossible to search through all the possible states before determining the move

### How can AI search algorithms be applied in this bot?
This AI bot deploys either of the two search algorithms: NegaMax (a variance of the MiniMax algorithm) and Monte Carlo Tree Search. These two search algorithms does a partial search on all the possible steps given a state of the board and return the best move it can find. While one is breadth-first search (Monte Carlo Tree Search), while the other is depth-first search (NegaMax), both are able to search through a large number of board states and determine the best step given the steps simulated.

Using the tree search method that is implemented in both Monte Carlo Tree Search and NegaMax can allow the computer to be methodological in approaching the possible steps, simulate many steps ahead of the current state, and retrace the steps that lead to the best outcome.


----

## Connect Four Class
Establishing the main methods for createing, playing and verifying the game. The state of the board is representedas a 7x6 matrix, with "1" indicating disc from Player 1, and "2" indicating disc from Player 2

In [1]:
class ConnectFour():
    
    ROW = 6
    COLUMN = 7
    INITIAL_STATE = [[],] * COLUMN
    PLAYERS = (1,2)
    GOAL = 4
    VALUE_WIN = 1
    VALUE_LOSE = -1
    VALUE_DRAW = 0
    
    #For Negamax algorithm implementation
    STREAK_WEIGHT = [1, 8, 128, 99999]
    evaluated = {}
    
    def __init__(self,row=ROW,column=COLUMN,state=INITIAL_STATE,player=PLAYERS,goal=GOAL):
        self.row = row
        self.column = column
        self.state = state
        self.player = player
        self.goal = goal
    
    def legal(self,state,action):
        if len(self.state[action]) >= self.row:
            raise Exception("Invalid action: columnn is full!")
        else:
            return len(self.state[action]) < self.row
        
    def actions(self, state):
        return [i for i in xrange(len(state)) if self.legal(state, i)]
    
    def update_state(self, state, action, player):
        if not self.legal(state, action):
            raise Exception('Illegal action')
        newstate = []
        for index, column in enumerate(state):
            if index == action:
                newstate.append(column + [player])
            else:
                newstate.append(column)
        #newstate[len(state)-1] = player
        return newstate
    
    def streak(self,state,player,start,delta,length=0):
        row, column = start
        if row < 0 or column < 0:
            return False
        
        try:
            piece = state[column][row]
        except IndexError:
            return False
        
        if piece != player:
            return False
        
        # Current slot is owned by the player
        length += 1
        if length == self.goal: # Streak is already long enough
            return True
        # Continue searching, 
        drow, dcolumn = delta
        
        return self.streak(state,player,(row + drow, column + dcolumn),delta,length)        
            
    
    def outcome(self, state, player):
        for col, column in enumerate(state):
            for row, marker in enumerate(column):
                if any((
                    self.streak(state, marker, (row, col), (1, 0)),  #check upwards
                    self.streak(state, marker, (row, col), (0, 1)),  #check rightward 
                    self.streak(state, marker, (row, col), (1, 1)),  #check right diagonal
                    self.streak(state, marker, (row, col), (1, -1)), #check left diagonal
                )):
                    # A winner was found
                    if marker == player:
                        return self.VALUE_WIN
                    else:
                        return self.VALUE_LOSE
        # No winner was found
        return self.VALUE_DRAW
    
    def terminal(self, state):
        if all([len(column) == self.row for column in state]):  #when all columns are full
            return True
        if self.outcome(state, self.player[0]) != self.VALUE_DRAW:  #when there is a winner
            return True
        return False  #the board still have space and the game is still on
    
    def next_player(self, player):
        if player not in self.player:
            raise Exception('Invalid player')
        index = self.player.index(player)
        if index < 1:
            return self.player[index + 1]
        else:
            return self.player[0]
    
    def pretty_state(self, state, escape=False):
        output = ''
        for j in range(self.column):
            output += ' ' + str(j)
        output += ' '
        if escape:
            output += '\\n'
        else:
            output += '\n'
        #boardState = state[1:self.column-1]
        i = self.row - 1
        while i >= 0:
            for column in state:
                if len(column) > i:
                    output += '|' + str(column[i])
                else:
                    output += '| '
            output += '|'
            if escape:
                output += '\\n'
            else:
                output += '\n'
            i -= 1
        return output
    
    
    #For Negamax algorithm implementation
    def streak_eval(self,state,player,start,delta,length=0,max_length = 0):
        row, column = start
        if row < 0 or column < 0:
            return max_length
        
        try:
            piece = state[column][row]
        except IndexError:
            return max_length
        
        if piece != player:
            length = 0
        
        # Current slot is owned by the player
        length += 1
        if length > max_length: # if current streak is longer than the max_length
            max_length = length
        # Continue searching, 
        drow, dcolumn = delta
        
        if row == self.row and column == self.column:
            return max_length
        
        return self.streak_eval(state,player,(row + drow, column + dcolumn),delta,length,max_length)
    
    def evaluate(self,state,player):
        curPlayer = player
        curPlayerValue = 0
        otherPlayerValue = 0
        
        for col, column in enumerate(state):
            for row, marker in enumerate(column):
                max_length_up = self.streak_eval(state, marker, (row, col), (1, 0))  #check upwards
                max_length_right = self.streak_eval(state, marker, (row, col), (0, 1))  #check rightward 
                max_length_diaright = self.streak_eval(state, marker, (row, col), (1, 1))  #check right diagonal
                max_length_dialeft = self.streak_eval(state, marker, (row, col), (1, -1)) #check left diagonal
                if marker == curPlayer:
                    max_length = max(max_length_up,max_length_right,max_length_diaright,max_length_dialeft)
                    curPlayerValue += self.STREAK_WEIGHT[max_length-1]
                    #print "Player two value updated: max_length:{} player_2_value:{}\n".format(max_length, player_2_value)
                else:
                    max_length = max(max_length_up,max_length_right,max_length_diaright,max_length_dialeft)
                    otherPlayerValue += self.STREAK_WEIGHT[max_length-1]
                    #print "Player one value updated: max_length:{} player_1_value:{}\n".format(max_length, player_1_value)
        
        difference = curPlayerValue - otherPlayerValue 
        return difference

In [2]:
#Implemented from my past assignment
#Use to flatten the state of the 2d board so to allow hashing
def flatten(board):
    # if it's nested lists, flatten them. I do this with list comprehension taking each tile at a time from each sublist
    if type(board[1])==list:
        board = [item for sublist in board for item in sublist] 
    # else, it should be a list of ints or floats
    elif type(board[1])==int or type(board[1])==float: 
        board = board
    # if it's neither, it's a wrong input and will raise an error.
    else:
        raise ValueError("Class 'PuzzleNode' got values that are not a sublist of ints nor a flat list of ints.")
    return board

### Class Methods
#### __init__()
Create a board given the board size (row and column), the current state (or a state with no disc), the current player (default is the first player) and the goal of the game (default is 4, as in connecting four of the same-colored disc)

#### legal()
Determine whether the action parsed can be performed given the current state (an action cannot be performed if the column is full). Return **TRUE** if the action can be performed

#### actions()
Given the state, return a list of available actions that the current player can perform

#### update_state()
Update the state of the board when given a move and the player. Add the player number to the representation of the board

#### streak()
Auxilary function to evaluate whether the board has a player that connected four. It is called on the *outcome()* function

#### outcome()
Returns the value of the outcome given the player and the state: 1 = the player won; -1 = the player lose; 0 = they draw

#### terminal()
Check if the board is full, or there is a player given the current state. Returns **TRUE** either there is a winner, or the game draws

#### next_player()
Given the current player, return the number of the next player so to determine legal moves and update the state once hte player has moved

#### pretty_state()
Given the board, show a visual version of the current state of the board

#### streak_eval() and evaluate()
Auxilary functions for NegaMax search algorithm implementation (these two functions will be discussed in the later section)


------

## Node Class
Establishing the leaf nodes of the search tree, given the current state, the action played, the player playing, the game the player is playing, the parent of the leaf node.

When using NegaMax, it will also specify the maximum depth that the algorithm will branch into

In [3]:
from math import sqrt, log
import random

class MonteCarloNode():
    
    def __init__(self,state,action,player,cur_game=None,parent=None,max_depth=None):
        self.game = game or parent.game
        
        #About the parents and children of the node
        self.parent = parent
        self.children = dict.fromkeys(self.game.actions(state))
        
        #About the move of the node
        self.state = state
        self.action = action
        
        #About the player and the reward of the node
        self.player = player
        self.visit = 0.0
        self.value = 0.0
        self.weight = 0.0
        
        #For Negamax algorithm implementation
        self.nega_value = 0
        self.max_depth = max_depth
        self.flatten_state = flatten(self.state)
        self.stateId = hash(tuple(self.flatten_state))
    
    def __hash__(self):
        if self.stateId is None:
            self.stateId = hash(tuple(self.flatten_state))
        return self.stateId
    
    def _tree_policy(self):
        bestNode = self
        while not bestNode.game.terminal(self.state):
            if not bestNode._fully_expanded():
                return bestNode._expand()
            else:
                bestNode = bestNode._best_child()
        return bestNode
    
    def _expand(self):
        try:
            expAction = self.children.keys()[self.children.values().index(None)]
        except ValueError:
            raise Exception('Node is already fully expanded')
        
        expState = self.game.update_state(self.state,expAction,self.player)
        expPlayer = self.game.next_player(self.player)
        
        child = MonteCarloNode(expState,expAction,expPlayer,self.game,self)
        self.children[expAction] = child
        return child
    
    def _fully_expanded(self):
        return not None in self.children.values()
    
    def _weight(self):
        if self.visit == 0:
            return 0.0
        else:
            return self.value/self.visit
    
    def _default_policy(self,player):
        game = self.game
        curState = self.state
        curPlayer = self.player
        while not game.terminal(curState):
            curAction = random.choice(game.actions(curState))
            curState = game.update_state(curState,curAction,curPlayer)
            curPlayer = game.next_player(curPlayer)
        reward = game.outcome(curState,player)
        return reward
        
    def _backup(self,reward,budget,pntlevel=0):
        backupNode = self
        search_depth = 1
        while not backupNode is None:
            backupNode.value += reward
            backupNode.visit += 1
            search_depth += 1
            backupNode = backupNode.parent
            
            if pntlevel != 0:
                if budget % 100 == 0:
                    print backupNode
        
        return search_depth
            
    def _best_child(self,c=1/sqrt(2)):
        return max(self.children.values(), key=lambda x: x._search_weight(sqrt(c)))
    
    def _search_weight(self, c):
        return self._weight() + c * sqrt(2 * log(self.parent.visit) / self.visit)

### Class Functions
### Functions for NegaMax 
#### __hash__()
Hash the leaf node using the current state of the board so the leaf node and its content can be retrieve in the future (used in NegaMax search algorithm)

### Functions for Monte Carlo Tree Search
#### _tree_policy()
**Tree policy** of the Monte Carlo Tree Search (explanation in the later section)

#### _expand()
Given the state and its available action, randomly select one and create a child node based on the new state after exercising that action

#### _fully_expand()
Return **TRUE** if the current node is fully populated with child nodes

#### _default_policy()
**Default policy** of the Monte Carlo Tree Search (explanation in the later section)

#### _weights()
Assign weights to the node based on this formula: reward/visit  
Reward: +1 if the player wins, -1 if the player loses, 0 if its draw; accumulated based on outcome of the child node)  
Visit: number of times this node has been visited (accounting for repeated visits if it is not a leaf node, but an intermediate node)

#### _search_weight()
A search function that assigns weights to nodes not only just by the reward/visit formula, but also account for the number of times the node has been visited. Used in the UCT variant of the Monte Carlo Tree Search (explanation in the later section)

#### _backup()
**Backward propagation** of the Monte Carlo Tree Search (Explanation in the later section)

#### _best_child()
Return the child node with the best search weights given by the *_search_weight()* function


-------

## Nega-Max for Connect Four

### What is the Nega-max Search Algorithm
NegaMax search algorithm is a variation of the MiniMax search algorithm. The MiniMax search algorithm seeks to maximize the utility function when its the algorithm's turn to play, the minimize the utility when its the other player's turn (hence maximizing the algorithm's utility).

The algorithm simulates the possible steps by alternating players, until it reaches a terminating stage (win, lose or draw)

NegaMax has the same aim of maximizing the algorithm's utility during its turn, and minimize the utility during the other player's turn. But it is a simplified version of MiniMax, utilizing the fact that in connect four, the outcome is dichotomic, meaning that the minimum utility is the negative (losing has the utility of -N) of the maximum utility (wining has the utility of N)

### Inner Working of the Nega-max Search Algorithm

In [4]:
#returns the best node (and best score) for a given state
def negamax(game,state,player,depth=0,max_depth=4,parent=None,action=None,s_steps=0,s_depth=0):
    current = MonteCarloNode(state,action,player,game,parent,max_depth)
    
    search_step = s_steps
    search_depth = s_depth
    
    search_step += 1
    
    if current in game.evaluated:
        return (current, current.nega_value,search_step,search_depth)
    
    if depth == current.max_depth:
        score = current.game.evaluate(current.state,current.player)
        game.evaluated[current] = score
        return (current, score,search_step,search_depth)
    
    best_score = float('-inf')
    best_node = None
    
    for nextAction in game.actions(current.state):
        nextState = game.update_state(current.state,nextAction,current.player)
        
        if game.terminal(nextState):
            outcome = game.outcome(nextState,player)
            if outcome == game.VALUE_WIN:
                current.nega_value = 99999
            elif outcome == game.VALUE_LOSE:
                current.nega_value = -99999
            else:
                current.nega_value = 0
            best_subscore = current.nega_value
            nextPlayer = game.next_player(current.player)
            #create MonteCarloNode to retrieve the action at this level
            best_subnode = MonteCarloNode(nextState,nextAction,nextPlayer,game,current,max_depth)
        
        else:
            nextPlayer = game.next_player(current.player)
            newDepth = depth + 1
            
            (best_subnode, best_subscore, search_step, search_depth) = negamax(game,nextState,nextPlayer,newDepth,max_depth,current,nextAction,search_step,search_depth)
            best_subscore *= -1
        
        if best_subscore > best_score:
            best_score = best_subscore
            best_node = best_subnode
    
    if best_node is None:
        best_score = game.evaluate(current.state,player)
    
    search_depth += 1
    
    game.evaluated[current] = best_score
    
    return (best_node, best_score, search_step, search_depth)

NegaMax works with a recursive call. At every call, it will create a leaf node based on a legal action. If the state of the leaf node is not a win, lose or draw, it will branch again and form a child node with another action.  

Procedure for NegaMax Search Algorithm:
1. Create a leaf node based on the state and player parsed when Negamax is called  
2. If the node has been evaluated, return the evaluated node and its utility  
3. If the current depth of search reaches the maximum depth designated, and the game has not terminate yet, the utility will be given as the value of a heuristic function (more about the heuristic function in the next subsection)  
4. Update the state based on the action and state of the current child node, and evaluate whether the game has terminated yet  
4.1 If the game has terminated, assign the utility of the current child (Win = 99999; Lose = -99999; Draw = 0)
4.2 If the game has not terminated, repeat step 1-4, but with the next player
5. Assign the best_utility
6. Store the current board in the evaluated dictionary for future retrieval
7. Return the node with the best utility, so that we can retrieve the action needed to achieve the best utility

### What if the game does not terminate after the algorithm reaches maximum search depth?
As mentioned above, not every path will lead to a terminal state when the maximum search depth is reached. In that case, we cannot assign a simplistic win-lose-draw utility to that particular leaf node. 

Here we introduce a heuristic that assign utility based on the number of chains in the current state for each player. The detailed implementation can be found in the *evaluate()* function under the *ConnectFour* class

The heuristic function counts the number of chains with different length (horizontal, vertical or diagonal) for each player. Chains with different length will be assigned different scores (length of 1:1; length of 2:8; length of 3:128, length of 4:99999). The utility of the node will be the difference between the score of the algorithm as a player, and the score of the other player (computer or human).

The underlying principle is that a longer chain implies higher chance of winning. Therefore different lengths of chain will have different score weightings. The difference between the scores will signal which player has a higher chance of winning if we continue to simulate the game.

--------

## Simplistic Monte Carlo Tree Search
The limitation of the NegaMax search algorithm (or any variation of the MiniMax search algorithm) is that the number of search steps is usually very large to run through a large amount of simulation in order to obtain the best move. A way to mitigate this problem is using the evaluate heuristic introduced above.

Another search algorithm that can mitigate the problem with MiniMax when it encounters high branching factor is Monte Carlo Tree Search. Monte Carlo Tree Search chooses the node with the best result (wins/game simulated) and simulate the rest game based on the action of that best node. Each time it simulates the game, it will reach a conclusion (win, lose or draw). It will then update the win/simulation ratio for all of its parents.

The benefits of Monte Carlo Tree Search over NegaMax algorithm in theory are:  
1. It can simulate more full games than NegaMax, so to provide a better estimate of the end result given a move
2. Every search is a greedy decision: selecting the node with the best outcome currently to explore. It saves a lot of search space compared to NegaMax's depth first search regardless of the outcome of the path

### Implementation of Monte Carlo Tree Search

In [5]:
import Queue, random

#returns an explored Monte Carlo Tree 
def mcts(game,state,player,budget):
    root = MonteCarloNode(state,None,player,cur_game=game,parent=None)
    #FIFO queue to store all the frontier nodes
    expQueue = Queue.Queue()
    expQueue.put(root)
    
    for i in xrange(budget):
        if expQueue.qsize == 0:
            break
        
        current = expQueue.get()
        if current.parent is not None:
            current.parent.children[current.action] = current
        
        #Tree Policy: Expanding the current node to children node
        for action in game.actions(current.state):
            childAction = action
            childPlayer = game.next_player(current.player)
            childState = game.update_state(current.state,childAction,childPlayer)
            curChild = MonteCarloNode(childState,childAction,childPlayer,cur_game=current.game,parent=current)
            expQueue.put(curChild)
       
       #Default Policy: Run simulation till the end of game to obtain reward
        curState = current.state
        curPlayer = current.player
        game = current.game
        while not game.terminal(curState):
            curAction = random.choice(game.actions(curState))
            curState = game.update_state(curState,curAction,curPlayer)
            curPlayer = game.next_player(curPlayer)
        reward = game.outcome(curState,player)
        
        #Back propagation: Propagate the reward result back to the root node
        backupNode = current
        while backupNode is not None:
            backupNode.value += reward
            backupNode.visit += 1
            backupNode.weight = backupNode.value/backupNode.visit
            backupNode = backupNode.parent

    bestChild = max(root.children.values(), key=lambda c: c.weight)
    return bestChild.action

There are **four steps** to Monte Carlo Tree Search:
#### Selection
The algorithm will select the node with the highest win/simiulation value to explore. If there is a tie, it will randomly select the node among the nodes that are tied. This process will go on until we reach a leaf node with no child.
#### Expansion
Once the path reaches a child-less node, it will expand and create a child node. The position (in this case, the action the child will take) is randomly chosen. The new child node will be added to the Monte Carlo Tree.
#### Simulation
Once a new, randomly chosen child is created, the algorithm will simulate the rest of the game and return an outcome (win, lose or draw). Because a full game have at most 47 steps, each simulation runs a full game. But for games with more average steps (e.g. chess), the outcome will be a estimation similar to **evaluate** in NegaMax.
#### Back propagation
Once there is an outcome to the simulated game, the algorithm will update the win/simulated value of all the nodes on the path. This will ensure that, when the algorithm is evaluating on the best move to make at one moment, it accounts for all the subsequent simulations given that move.  

----------------------------

The algorithm is configured to stop after any desired number of simulations (budget). Once the algorithm stops, it will look at the children nodes of the root, and determine the best node based on the win/simulated value, and return the action of the best child node.

## UCT Variant of Monte Carlo Tree Search
The problem with simple Monte Carlo Tree Search is that it is at a tug of war with two different goals: exploitation and exploration. 

On one hand, it wants to exploit the child node of the root with the best win/simulation value, going deeper into the branches of that node. On the other hand, it needs to explore other actions that may give a higher win/simulation value than the ones explored. The simple implementation exploits the child node with the best win/simulation, until there is enough loses to render it suboptimal. However, it may miss the exploration of other nodes that may later on yield better win/simulation ratio

The UCT variant of Monte Carlo Tree Search that tries to balance the exploitation and exploration goals of the search algorithm by assigning a search weight to each node based on both the win/simulation ratio and the number of times the node has been visited.

### UCT Search Weights

In [ ]:
def _search_weight(self, c):
    return self._weight() + c * sqrt(2 * log(self.parent.visit) / self.visit)

A child node *j* is selected to maximize the UCT search weight, given by this formula: 

[math formula for UCT]

If the child node is not visited, the search weight would be infinity, so that previously unvisited children are searched first. The constant *Cp* determines whether the Monte Carlo Tree Search bias towards exploitation (if *Cp* is small) or exploration (if *Cp* is large). With a reward of (1,-1), the optimal *Cp* is 1/sqrt(2), given by Kocsis and Szepesvari.

### Implementation Monte Carlo Tree Search with UCT
The UCT variant of Monte Carlo Tree Search is identical as the simplistic Monte Carlo Tree Search, except the best child is selected based on the highest UCT search weight, not the win/simulation ratio.

The following implementation replaced the in-function selection, expansion, simulation and back propagation with MonteCarloNode class functions. Other than that, it is identical to the previous Monte Carlo Tree Search.

In [6]:
def mcts_uct(game,state,player,budget,c=(1/sqrt(2))):
    root = MonteCarloNode(state,None,player,cur_game=game,parent=None)
    computerWin = 0.0
    originalBudget = budget
    
    search_step = max_search_depth = 0
    
    while budget:
        child = root
        child = child._tree_policy()
        search_step += 1
        reward = child._default_policy(player)
        search_depth = child._backup(reward,budget)
        if search_depth > max_search_depth:
            max_search_depth = search_depth
        budget-=1
     
    bestChild = root._best_child(c)
    return bestChild.action, search_step, max_search_depth

---------

## Negamax vs Monte Carlo Tree Search
To evaluate the practical performance of the two major search algorithms. I built a simulation and allow a NegaMax bot play against a UCT-Monte Carlo Tree Search bot. The simulation plays 50 games.

The metrics measured after the simulation for both bots are: 
* Search depth (per move)
* branching factor (per move)
* search time (per move)
* win percentage (over 100 games)

In [ ]:
import time
from numpy import mean


noOfSim = 50
originalNoOfSim = noOfSim
gameCount = 1

players = ['','NegaMax','MCTS-UCT']
mctsWin = 0.0
negaWin = 0.0
avgNegaSearchStep = []
avgNegaSearchDepth = []
avgNegaRunTime = []
avgMCTSSearchStep = []
avgMCTSSearchDepth = []
avgMCTSRunTime = []

game = ConnectFour()

while noOfSim > 0:
    currentState = [[],[],[],[],[],[],[]]
    game.state = currentState
    player = noOfSim%2+1
    currentPlayer = player
    
    allNegaSearchStep = []
    allNegaSearchDepth = []
    allNegaRunTime = []
    allMctsSearchStep = []
    allMctsSearchDepth = []
    allMctsRunTime = []
    
    while not game.terminal(currentState):
        if currentPlayer == 1:
            start_time = time.time()
            negaRunTime = 0.0
            negaNode, negaScore, negaSearchStep, negaSearchDepth = negamax(game,currentState,currentPlayer,max_depth=4)
            negaRunTime = time.time()-start_time
            negaAction = negaNode.action
            
            if game.legal(currentState,negaAction):
                currentState = game.update_state(currentState,negaAction,currentPlayer)
                currentPlayer = game.next_player(currentPlayer)
            
            allNegaSearchStep.append(negaSearchStep)
            allNegaSearchDepth.append(negaSearchDepth)
            allNegaRunTime.append(negaRunTime)
        
        else:
            start_time = time.time()
            mctsRunTime = 0.0
            mctsAction, mctsSearchStep, mctsSearchDepth = mcts_uct(game,currentState,currentPlayer,200)  #using the UCT variant of MCTS algorithm
            mctsRunTime = time.time()-start_time
            
            if game.legal(currentState,mctsAction):
                currentState = game.update_state(currentState,mctsAction,currentPlayer)
                currentPlayer = game.next_player(currentPlayer)
            
            allMctsSearchStep.append(mctsSearchStep)
            allMctsSearchDepth.append(mctsSearchDepth)
            allMctsRunTime.append(mctsRunTime)
    
    avgNegaSearchStep.append(mean(allNegaSearchStep))
    avgNegaSearchDepth.append(mean(allNegaSearchDepth))
    avgNegaRunTime.append(mean(allNegaRunTime))
    
    avgMCTSSearchStep.append(mean(allMctsSearchStep))
    avgMCTSSearchDepth.append(mean(allMctsSearchDepth))
    avgMCTSRunTime.append(mean(allMctsRunTime))
    
    outcome = game.outcome(currentState,player)
    if outcome > 0:
        print "{} won the {}-th game.".format(players[player],gameCount)
        if player == 1: negaWin += 1
        else: mctsWin += 1
    elif outcome < 0:
        if player == 1:
            print "{} won the {}-th game.".format(players[player+1],gameCount)
            mctsWin += 1
        else:
            print "{} won the {}-th game.".format(players[player-1],gameCount)
            negaWin += 1
    else:
        print "{}-th game is draw.".format(gameCount)
    
    gameCount += 1
    noOfSim -= 1
    
#Results
negaWinRatio = negaWin/originalNoOfSim
mctsWinRatio = mctsWin/originalNoOfSim
print "\n==============================================================="
print "Here are the results for the simulation"
print "                                   NegaMax   vs   MCTS-UCT"
print "Win Ratio:                          %.2f            %.2f" %(negaWinRatio,mctsWinRatio)
print "Runtime (sec/move):                %.3f           %.3f" %(mean(avgNegaRunTime),mean(avgMCTSRunTime))
print "Average Search Step (per move):    %.3f        %.3f" %(mean(avgNegaSearchStep),mean(avgMCTSSearchStep))
print "Average Search Depth (per move):   %.3f         %.3f" %(mean(avgNegaSearchDepth),mean(avgMCTSSearchDepth))
print "\n===============================================================\n"

MCTS-UCT won the 1-th game.
MCTS-UCT won the 2-th game.
MCTS-UCT won the 3-th game.
MCTS-UCT won the 4-th game.
MCTS-UCT won the 5-th game.
MCTS-UCT won the 6-th game.
NegaMax won the 7-th game.
MCTS-UCT won the 8-th game.
MCTS-UCT won the 9-th game.
MCTS-UCT won the 10-th game.
MCTS-UCT won the 11-th game.
MCTS-UCT won the 12-th game.
MCTS-UCT won the 13-th game.
MCTS-UCT won the 14-th game.
MCTS-UCT won the 15-th game.
MCTS-UCT won the 16-th game.
MCTS-UCT won the 17-th game.
NegaMax won the 18-th game.
MCTS-UCT won the 19-th game.
MCTS-UCT won the 20-th game.
MCTS-UCT won the 21-th game.
MCTS-UCT won the 22-th game.
MCTS-UCT won the 23-th game.
MCTS-UCT won the 24-th game.
MCTS-UCT won the 25-th game.
MCTS-UCT won the 26-th game.
NegaMax won the 27-th game.
MCTS-UCT won the 28-th game.
MCTS-UCT won the 29-th game.
NegaMax won the 30-th game.
MCTS-UCT won the 31-th game.
MCTS-UCT won the 32-th game.
MCTS-UCT won the 33-th game.
MCTS-UCT won the 34-th game.
MCTS-UCT won the 35-th game

### Analysis of the simulation
Based on the wins, Monte Carlo Tree Search performs much better than NegaMax. It also has a smaller runtime and smaller tree to search for every move.

The reason behind the small search tree is that MCTS is a breadth-first search algorithm, while NegMax is a depth-first search algorithm. MCTS starts from the roots, chooses a child node, runs a full game simulation, and then usually chooses another chld node on the same level (because of the UCT search weights that balances on exploration). NegaMax chooses follows a pathh of the best child down the tree, until it either hits the search depth or the game ends. Therefore the search depth of NegaMax is significant higher than MCTS.

A potential explanation on the better performance of MCTS lies on better estimate of the utility of the move and seeking for global optima. As mentioned above, MCTS runs more full game simulation, which provides better estimate of the utility of the move. However, becasue the moves of the simulation are random. The UCT exploration balancing make sure that we explore more variations of moves to attempt to reach a global optima in terms of win/simulation. These two factors made MCTS a much more optimized algorithm in terms of search time and outcome.

An interesting phenomenon observed is that, as the simulation progress, the number of wins by NegaMax increased slightly (1 for the first 10 games, 1 in the second 10 games, 2 in the third 10 games, 0 in the forth 10 game, 3 in the fifth 10 game). This phenomenon could be random, or could be that the algorithm remembers different states from previous games and skip through some steps in a full game tree, hence being able to simulate more complete games as it plays more.

----

## Actual Gameplay
Running the code below allows you to play against either one of the AIFour Connect Four AI algorithms

In [ ]:
game = ConnectFour()
currentState = game.state
player = 1

print "There are two variation of AI algorithm you can play with:"
print "Negamax (negamax): A simpler variation of Minimax algorithm"
print "UCT Variant of MCTS (monteuct): MCTS with the UCT search-weighting function"
selection = str(raw_input("\nWhich AI algorithm do you want to play against (negamax,monteuct): "))

print "\nYou are playing with an AI agent that uses {}".format(selection)
print "\n====================================================\n"
    
currentPlayer = player
while not game.terminal(currentState):
    if currentPlayer == 1:
        humanAction = int(raw_input("Choose a column: "))
        if game.legal(currentState,humanAction):
            currentState = game.update_state(currentState,humanAction,currentPlayer)
            currentPlayer = game.next_player(currentPlayer)
            print "You played column {}".format(humanAction)
    else:
        if selection == "monteuct":
            computerAction,searchStep,searchDepth = mcts_uct(game,currentState,currentPlayer,2000)  #using the UCT variant of MCTS algorithm
        elif selection == "negamax":
            computerNode, computerScore,searchStep,searchDepth = negamax(game,currentState,currentPlayer,max_depth=4)
            computerAction = computerNode.action
        else:
            raise ValueError("Invalid Choice of the AI algorithm")
        
        if game.legal(currentState,computerAction):
            currentState = game.update_state(currentState,computerAction,currentPlayer)
            currentPlayer = game.next_player(currentPlayer)
            print "The computer played column {}".format(computerAction)
    
    currentBoard = game.pretty_state(currentState)
    print currentBoard
    
outcome = game.outcome(currentState,player)
if outcome > 0:
    print "You won!"
elif outcome < 0:
    print "You lose!"
else:
    print "Draw!"

There are two variation of AI algorithm you can play with:
Negamax (negamax): A simpler variation of Minimax algorithm
UCT Variant of MCTS (monteuct): MCTS with the UCT search-weighting function


## References
GUI for connect four: https://github.com/uroshekic/connect-four/blob/master/ConnectFour.py  

Nega-max Algorithm: http://blog.gamesolver.org/solving-connect-four/03-minmax/  
Explanation of Nega-max Algorithm: http://www.hamedahmadi.com/gametree/#negamax

Monte Carlo Tree Search Algorithm: http://mcts.ai/pubs/mcts-survey-master.pdf  
Sample Implementation of Monte Carlo Tree Search: https://jeffbradberry.com/posts/2015/09/intro-to-monte-carlo-tree-search/
More Monte Carlo Implementation: https://mattschmoyer.com/connect-four-ai/  